In [2]:
import numpy as np
import pandas as pd
import sys
!{sys.executable} -m pip install plotly
!{sys.executable} -m pip install cufflinks
!{sys.executable} -m pip install wordcloud
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from wordcloud import WordCloud,STOPWORDS

%matplotlib inline


plt.rcParams['figure.figsize'] = (6, 4)
plt.style.use('ggplot')
%config InlineBackend.figure_formats = {'png', 'retina'}

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.2.9_1/libexec/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.2.9_1/libexec/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.2.9_1/libexec/bin/python -m pip install --upgrade pip' command.


In [3]:
book_data = pd.read_csv('books.csv')
book_data = book_data.dropna()
book_data.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [4]:
book_data.columns

Index(['book_id', 'goodreads_book_id', 'best_book_id', 'work_id',
       'books_count', 'isbn', 'isbn13', 'authors', 'original_publication_year',
       'original_title', 'title', 'language_code', 'average_rating',
       'ratings_count', 'work_ratings_count', 'work_text_reviews_count',
       'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5',
       'image_url', 'small_image_url'],
      dtype='object')

In [5]:
# Dropping irrelevant columns
book_data = book_data.drop(columns=['goodreads_book_id', 'best_book_id', 'work_id', 'isbn', 'isbn13', 'title','work_ratings_count','work_text_reviews_count', 'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url','small_image_url'])
book_data.head()

,book_id,books_count,authors,original_publication_year,original_title,language_code,average_rating,ratings_count
0,1,272,Suzanne Collins,2008.0,The Hunger Games,eng,4.34,4780653
1,2,491,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,eng,4.44,4602479
2,3,226,Stephenie Meyer,2005.0,Twilight,en-US,3.57,3866839
3,4,487,Harper Lee,1960.0,To Kill a Mockingbird,eng,4.25,3198671
4,5,1356,F. Scott Fitzgerald,1925.0,The Great Gatsby,eng,3.89,2683664


In [6]:
rating_data = pd.read_csv('ratings.csv')
rating_data.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [7]:
# Calculating mean rating per user - Why?
MRPU = rating_data.groupby(['user_id']).mean().reset_index()
MRPU['mean_rating'] = MRPU['rating']

MRPU.drop(['book_id','rating'],axis=1, inplace=True)
MRPU.head()

,user_id,mean_rating
0,1,3.589744
1,2,4.415385
2,3,1.736264
3,4,3.768657
4,5,4.040000


In [8]:
# Merging mean rating per user with ratings dataframe
rating_data = pd.merge(rating_data, MRPU, on=['user_id', 'user_id'])
rating_data.head()

,user_id,book_id,rating,mean_rating
0,1,258,5,3.589744
1,1,268,3,3.589744
2,1,5556,3,3.589744
3,1,3638,3,3.589744
4,1,1796,5,3.589744


In [9]:
# Looking at count of ratings per rating
rating_data['rating'].value_counts().iplot(kind='bar',
                                          xTitle='Rating',
                                          yTitle='Count of Ratings',
                                          title='Rating vs Rating Frequency',
                                          color='red')

In [10]:
# # Visualizing most popular authors and books using WordCloud
# stop_words=set(STOPWORDS)
# authors_string = " ".join(book_data['authors'])
# wc = WordCloud(width=800,height=500,mask=None,random_state=21, max_font_size=110,stopwords=stop_words).generate(authors_string)

In [11]:
# Dropping ratings lesser than the mean
rating_data = rating_data.drop(rating_data[rating_data.rating < rating_data.mean_rating].index)

In [12]:
#user2's favorite books
rating_data[rating_data['user_id']== 2].head()

,user_id,book_id,rating,mean_rating
118,2,260,5,4.415385
119,2,9296,5,4.415385
124,2,301,5,4.415385
125,2,2686,5,4.415385
126,2,3753,5,4.415385


In [13]:
#user10's favorites
rating_data[rating_data['user_id']== 10].head()

,user_id,book_id,rating,mean_rating
11110,10,883,4,3.75
11111,10,217,4,3.75
11112,10,103,5,3.75
11113,10,1506,4,3.75
11116,10,5,5,3.75


In [14]:
# Retaining only unique values
rating_data.shape

(3355311, 4)

In [15]:
len(rating_data['user_id'].unique())

53424

In [16]:
# Rename rating to user rating and merge to books dataframe
rating_data = rating_data.rename({'rating':'user_rating'}, axis='columns')
mergedata = pd.merge(book_data,rating_data,on=['book_id','book_id'])
mergedata.head()

,book_id,books_count,authors,original_publication_year,original_title,language_code,average_rating,ratings_count,user_id,user_rating,mean_rating
0,1,272,Suzanne Collins,2008.0,The Hunger Games,eng,4.34,4780653,9,4,3.446970
1,1,272,Suzanne Collins,2008.0,The Hunger Games,eng,4.34,4780653,28,5,4.008333
2,1,272,Suzanne Collins,2008.0,The Hunger Games,eng,4.34,4780653,32,5,3.983740
3,1,272,Suzanne Collins,2008.0,The Hunger Games,eng,4.34,4780653,40,4,3.844961
4,1,272,Suzanne Collins,2008.0,The Hunger Games,eng,4.34,4780653,31,4,3.504132


In [17]:
len(mergedata['book_id'].unique())

7860

In [18]:
len(book_data['book_id'].unique())

7860

No duplicates

In [19]:
# Crosstab - Compute a simple cross tabulation of two (or more) factors. 
# By default computes a frequency table of the factors unless an array of values and an aggregation function are passed.
user_book = pd.crosstab(mergedata['user_id'], mergedata['original_title'])
user_book.head()

original_title,,A Monster Calls,Animal Farm & 1984,"Burned (Burned, #1)",Call the midwife : a true story of the East End in the 1950s,"Carry On, Warrior: Thoughts on Life Unarmed",Carter Beats the Devil,Crown of Shadows,Daemon,Evil at Heart,...,鋼の錬金術師 1,雪国,青の祓魔師 1,青の祓魔師 2,高校デビュー 1,黑暗森林,黒執事 I [Kuroshitsuji I],黒執事 II [Kuroshitsuji II],엄마를 부탁해,채식주의자
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
user_book.shape

(53421, 7755)

In [ ]:
from sklearn.decomposition import PCA
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# data_rescaled = scaler.fit_transform(user_book)

# #99% of variance
# from sklearn.decomposition import PCA
# pca = PCA(n_components = 0.99)
# pca.fit(data_rescaled)
# reduced = pca.transform(data_rescaled)

# pca = PCA()
# pca.fit(user_book)
# print("Works PCA()!")
# plt.plot(np.cumsum(pca.explained_variance_ratio_))
# print("Explained PCA!")
# plt.xlabel('number of components')
# plt.ylabel('cumulative explained variance')

pca = PCA(n_components=3)
# Fit the model with X.
pca.fit(user_book)
# Apply dimensionality reduction to X.
pca_samples = pca.transform(user_book)
bs = pd.DataFrame(pca_samples)
bs.head()

In [ ]:
tocluster = pd.DataFrame(bs[[0,1,2]])

In [ ]:
plt.rcParams['figure.figsize'] = (10, 6)


fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(tocluster[0], tocluster[2], tocluster[1], color="blue")

plt.title('PCA Reduced Data Points', fontsize=20)
plt.show()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

Sum_of_squared_distances = []
scores = []
K = range(1,17)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(tocluster)
    Sum_of_squared_distances.append(km.inertia_)

# for k in K:
#     kmeans = KMeans(n_clusters=k)
#     kmeans.fit(tocluster)
#     scores.append(silhouette_score(tocluster, kmeans.labels_))
    
# for i in K:
#     kmeans = KMeans(n_clusters=i)
#     kmeans.fit(tocluster)
#     print(kmeans)
#     #inertia_list[i] = kmeans.inertia_
#     scores.append(silhouette_score(tocluster, kmeans.labels_))

In [ ]:
# plt.plot(K, scores, 'bx-')
# plt.ylabel('Silhouette Score')
# plt.xlabel('k')
# plt.title('Silhouette method for optimal k')
# plt.show()

In [ ]:
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.ylabel('Sum of squared distances')
plt.xlabel('k')
plt.title('Elbow method for optimal k')
plt.show()

In [ ]:
# From silhoutte method, we choose number of clusters to be 4
clusterer = KMeans(n_clusters=3,random_state=30).fit(tocluster)
centers = clusterer.cluster_centers_
c_preds = clusterer.predict(tocluster)

print(centers)

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.scatter(tocluster[0], tocluster[1], c = c_preds)
for ci,c in enumerate(centers):
    plt.plot(c[0], c[1], 'o', markersize=8, color='red', alpha=1)

plt.xlabel('x')
plt.ylabel('y')

plt.title('Clustered data points', fontsize=20)
plt.show()

In [ ]:
len(c_preds)

In [ ]:
user_book['cluster'] = c_preds
user_book.head()

In [ ]:
user_book.info()

In [ ]:
print(user_book[user_book['cluster']==0].drop('cluster', axis=1).mean())

In [ ]:
c0 = user_book[user_book['cluster']==0].drop('cluster',axis=1).mean()
c1 = user_book[user_book['cluster']==1].drop('cluster',axis=1).mean()
c2 = user_book[user_book['cluster']==2].drop('cluster',axis=1).mean()
#c3 = user_book[user_book['cluster']==3].drop('cluster',axis=1).mean()

In [ ]:
c0.head()

In [ ]:
c0.sort_values(ascending=False)

In [ ]:
c1.sort_values(ascending=False)[0:10]

In [ ]:
c2.sort_values(ascending=False)[0:10]